In [7]:
import pandas as pd
import requests
import re
import time
pd.set_option("display.max_columns", None)

In [8]:


def llamada_api(base_url, headers, params):
    data_pelis = []
    page = 1  
    
    while True:
       
        params['page'] = str(page)
        
        response = requests.get(base_url, headers=headers, params=querystring)
        if response.status_code != 200:
            print(f"Error al buscar data: {response.status_code} {response.reason}")
            break
        
        data = response.json()

        if 'results' in data:
            data_pelis.extend(data['results'])
            print(f"Page {page}: {len(data['results'])} pelis extraidas.")
        else:
            print("No 'results' en la respuesta. Finalizando bucle.")
            break

        
        if 'next' in data:
            page += 1  
            time.sleep(2)
        else:
            print("ultima pagina extraida.")
            break

    return data_pelis


url_base = "https://moviesdatabase.p.rapidapi.com/titles"


headers = {
    "X-RapidAPI-Key": "1db777625dmshdce6ed30bc0ec06p198b62jsn978c35c52050",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}


querystring= {
    "titleType": "movie",
    "year": "2011",
    "genre": "Drama"
}


resultados_pelis = llamada_api(url_base, headers, querystring)




Page 1: 10 pelis extraidas.
Page 2: 10 pelis extraidas.
Page 3: 10 pelis extraidas.
Page 4: 10 pelis extraidas.
Page 5: 10 pelis extraidas.
Page 6: 10 pelis extraidas.
Page 7: 10 pelis extraidas.
Page 8: 10 pelis extraidas.
Page 9: 10 pelis extraidas.
Page 10: 10 pelis extraidas.
Page 11: 10 pelis extraidas.
Page 12: 10 pelis extraidas.
Page 13: 10 pelis extraidas.
Page 14: 10 pelis extraidas.
Page 15: 10 pelis extraidas.
Page 16: 10 pelis extraidas.
Page 17: 10 pelis extraidas.
Page 18: 10 pelis extraidas.
Page 19: 10 pelis extraidas.
Page 20: 10 pelis extraidas.
Page 21: 10 pelis extraidas.
Page 22: 10 pelis extraidas.
Page 23: 10 pelis extraidas.
Page 24: 10 pelis extraidas.
Page 25: 10 pelis extraidas.
Page 26: 10 pelis extraidas.
Page 27: 10 pelis extraidas.
Page 28: 10 pelis extraidas.
Page 29: 10 pelis extraidas.
Page 30: 10 pelis extraidas.
Page 31: 10 pelis extraidas.
Page 32: 10 pelis extraidas.
Page 33: 10 pelis extraidas.
Page 34: 10 pelis extraidas.
Page 35: 10 pelis extra

In [10]:
resultados_pelis

[{'_id': '61e582368a5a6a599f418ebb',
  'id': 'tt0176694',
  'primaryImage': {'id': 'rm3068795392',
   'width': 400,
   'height': 571,
   'url': 'https://m.media-amazon.com/images/M/MV5BMTEwODljN2MtNjNhNC00NGIwLTk3YjgtODA5MzIzYjZiYWMwXkEyXkFqcGdeQXVyNTczMzEzMDY@._V1_.jpg',
   'caption': {'plainText': 'Az ember tragédiája (2011)',
    '__typename': 'Markdown'},
   '__typename': 'Image'},
  'titleType': {'text': 'Movie',
   'id': 'movie',
   'isSeries': False,
   'isEpisode': False,
   '__typename': 'TitleType'},
  'titleText': {'text': 'Az ember tragédiája', '__typename': 'TitleText'},
  'originalTitleText': {'text': 'Az ember tragédiája',
   '__typename': 'TitleText'},
  'releaseYear': {'year': 2011, 'endYear': None, '__typename': 'YearRange'},
  'releaseDate': {'day': 8,
   'month': 12,
   'year': 2011,
   '__typename': 'ReleaseDate'}},
 {'_id': '61e582f678c012aa3441b943',
  'id': 'tt0247643',
  'primaryImage': None,
  'titleType': {'text': 'Movie',
   'id': 'movie',
   'isSeries': Fal

In [11]:
def organizar_pelis(resultados):
    peli_data = []
    for i in resultados:
        # Extraer los datos necesarios
        peli_id = i['id']
        peli_type = i['titleType']['text'] if 'titleType' in i else 'Desconocido'
        peli_nombre = i['titleText']['text'] if 'titleText' in i else 'Desconocido'
        
        # Manejo seguro de releaseDate para evitar AttributeError
        release_date = i.get('releaseDate', None)  # Usa None como valor por defecto
        if release_date is None:  # Si releaseDate es None, usar valores por defecto
            peli_ano = 'Desconocido'
            peli_mes = 'Desconocido'
        else:
            peli_ano = release_date.get('year', 'Desconocido')
            peli_mes = release_date.get('month', 'Desconocido')
        
        # Crear el diccionario con los datos 
        movie_dict = {
            'Tipo': peli_type,
            'Nombre': peli_nombre,
            'Año de Estreno': peli_ano,
            'Mes de Estreno': peli_mes,
            'ID de la Película': peli_id
        }
        peli_data.append(movie_dict)
    
    # Convertir la lista de diccionarios en un DataFrame
    return pd.DataFrame(peli_data)

# Llamar a la función y crear el DataFrame
df_pelis = organizar_pelis(resultados_pelis)

# Mostrar el DataFrame resultante
df_pelis


,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película
0,Movie,Az ember tragédiája,2011,12,tt0176694
1,Movie,Los pájaros se van con la muerte,2011,6,tt0247643
2,Movie,Lives No Longer Ours,2011,None,tt0311448
3,Movie,The Wicker Tree,2011,7,tt0323808
4,Movie,Spy,2011,8,tt0372538
...,...,...,...,...,...
395,Movie,Journey,2011,4,tt13906900
396,Movie,Maryade Ramanna,2011,9,tt13906502
397,Movie,Bhadra,2011,8,tt13906572
398,Movie,Chennamma IPS,2011,7,tt13906610


In [12]:
# df_pelis.to_csv("drama_2011.csv", index=False, encoding='utf-8')